In [27]:
pip install streamlit chromadb langchain pypdf sentence-transformers arxiv


Note: you may need to restart the kernel to use updated packages.


In [28]:
import os
from pypdf import PdfReader
import chromadb
from langchain.embeddings import HuggingFaceEmbeddings

In [29]:
db_path="./chroma.db"
client=chromadb.PersistentClient(path=db_path)
collection=client.get_or_create_collection(name="research_papers")

In [30]:
embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [31]:
def extract_text_from_pdfs(pdf_folder="papers"):
    paper_texts=[]
    paper_filenames=[]

    for filename in os.listdir(pdf_folder):
        if filename.endswith(".pdf"):
            pdf_path=os.path.join(pdf_folder,filename)
            reader=PdfReader(pdf_path)
            text="\n".join([page.extract_text() for page in reader.pages])

            paper_texts.append(text)
            paper_filenames.append(filename)
    return paper_texts,paper_filenames
        

In [32]:
def embed_local_papers(pdf_folder="papers"):
    paper_texts,paper_filenames=extract_text_from_pdfs(pdf_folder)

    for i,text in enumerate(paper_texts):
        embedding = embedding_model.embed_query(text)  
        collection.add(
            ids=[f"paper_{i}"],
            documents=[text],
            metadatas=[{"filename":paper_filenames[i]}]
        )
    print(f" successfully embedded {len(paper_texts)} papers into chromadb")

In [33]:
# embed_local_papers()

In [34]:
def search_papers(query,top_k=5):
    query_embedding=embedding_model.embed_query(query)
    search_results=collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    relevant_passages=search_results["documents"]
    paper_filenames=[meta["filename"] for meta in search_results["metadatas"]]
    return relevant_passages,paper_filenames



In [36]:
query = "Recent advancements in neural architecture search"
passages, filenames = search_papers(query)

# Debugging: Check the types of passages and filenames
print(f"Type of passages: {type(passages)}")
print(f"Type of filenames: {type(filenames)}")

# Debugging: Print the elements of passages and filenames
print("Passages:")
for i, passage in enumerate(passages):
    print(f"Passage {i}: {passage[:500]}")  # Show first 500 chars

print("Filenames:")
for i, name in enumerate(filenames):
    print(f"Filename {i}: {name}")

# Original code
print("🔍 Most Relevant Passages:")
for passage in passages:
    print(passage[:500], "\n---\n")  # Show first 500 chars

print("📚 Relevant Papers:")
for name in filenames:
    print(name)

TypeError: list indices must be integers or slices, not str